In [1]:
import yaml
from pydub import AudioSegment
import numpy as np
import wave
import os
import numpy as np
import soundfile as sf
from scipy.signal import stft
import matplotlib.pyplot as plt
from tqdm import tqdm

/home/zaptos/miniconda3/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
directory = "data/babyslakh_16k"
listdir = os.listdir(directory)
listdir.sort()
tr_dicts = [{}]*len(listdir)
tr_dicts_2 = [{}]*len(listdir)

for i, tr in enumerate(tqdm(listdir)):
    tr_path = os.path.join(directory, tr)
    with open(os.path.join(tr_path, "metadata.yaml")) as meta:
        metadata = yaml.load(meta, Loader=yaml.Loader)
    file_inst = []
    for stem in metadata['stems'].keys():
        file_inst.append([stem + '.wav', metadata['stems'][stem]['inst_class']])
    file_inst = np.array(file_inst)
    file_inst = file_inst[[file_inst[:,0][i] in os.listdir(tr_path + '/stems') for i in range(len(file_inst))]]
    # combine all stems from the same instrument and track using soundfile
    for inst in np.unique(file_inst[:,1]):
        inst_files = file_inst[file_inst[:,1] == inst][:,0]
        inst_files = [os.path.join(tr_path, 'stems', inst_file) for inst_file in inst_files]
        inst_data = np.array([sf.read(inst_file)[0] for inst_file in inst_files])
        inst_data = np.sum(inst_data, axis=0)
        #update the dictionary with the combined data and the instrument name as the key
        tr_dicts[i].update({inst: inst_data})

    for inst in tr_dicts[i].keys():
        if inst in ['Bass', 'Guitar', 'Drums']:
            f, t, Zxx = stft(tr_dicts[i][inst], fs=16000, nperseg=256)
            tr_dicts_2[i].update({inst: np.abs(Zxx)})


# Plot the magnitude of the STFT for each instrument
#for i, tr in enumerate(tqdm(tr_dicts_2)):
#    for inst in tr.keys():
#        plt.pcolormesh(t, f, tr[inst], shading='gouraud')
#        plt.colorbar(label='Magnitude')
#        plt.title(f'STFT Magnitude: Track {i} Instrument: ' + inst)
#        plt.xlabel('Time [s]')
#        plt.ylabel('Frequency [Hz]')
#        plt.show()

100%|██████████| 20/20 [00:08<00:00,  2.43it/s]


In [ ]:
import yaml
from pydub import AudioSegment
import numpy as np
import wave
import os

old_directory = "babyslakh_16k"
new_directory = "new_babyslack"
try:
    os.mkdir(new_directory)
except:
    pass

for tr in os.listdir(old_directory):
    old_tr = os.path.join(old_directory, tr)
    new_tr = os.path.join(new_directory, tr)
    try:
        os.mkdir(new_tr)
    except:
        pass

    with open(os.path.join(old_tr, "metadata.yaml")) as meta:
        metadata = yaml.load(meta, Loader=yaml.Loader)

    file_inst = []
    for stem in metadata['stems'].keys():
        file_inst.append([stem + '.wav', metadata['stems'][stem]['inst_class']])

    file_inst = np.array(file_inst)
    file_inst = file_inst[[file_inst[:,0][i] in os.listdir(old_tr + '/stems') for i in range(len(file_inst))]]
    
    mix_path = os.path.join(new_tr, 'mix.wav')

    mix = AudioSegment.from_file(os.path.join(old_tr, 'mix.wav'))
    mix.export(mix_path, format='wav')

    unique_inst = np.unique(file_inst[:,1])
    for inst in unique_inst:
        inst_list = []
        inst_path = os.path.join(new_tr, inst + '.wav')
        for i in range(file_inst.shape[0]):
            if file_inst[:,1][i] == inst:
                inst_list.append(file_inst[:,0][i])
        if len(inst_list) > 1:
            sound1 = AudioSegment.from_file(os.path.join(old_tr + '/stems', inst_list[0]))
            for i in range(len(inst_list)-1):
                sound2 = AudioSegment.from_file(os.path.join(old_tr + '/stems', inst_list[i+1]))
                combined = sound1.overlay(sound2)
                sound1 = combined
            combined.export(inst_path, format='wav')
        else:
            sound = AudioSegment.from_file(os.path.join(old_tr + '/stems', inst_list[0]))
            sound.export(inst_path, format='wav')